In [2]:
import pandas as pd
import numpy as np

from eunjeon import Mecab

import torch
import torch.nn.functional as F
import torchtext.transforms as T

from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.optim import AdamW

from torch.utils.data import Dataset
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torchtext.nn import MultiheadAttentionContainer, InProjContainer, ScaledDotProduct
from torch.nn.utils.rnn import pad_sequence
from torchtext.data import get_tokenizer
from torchtext.vocab import vocab, Vocab
from torchtext.transforms import VocabTransform, ToTensor

from sklearn.model_selection import train_test_split

import os

import sentencepiece as spm

from transformers import BertTokenizer, BertForSequenceClassification

# Dataset

In [3]:
dataset = pd.read_csv(os.path.join('..','mbti_y', 'mbti_clean.csv'))

In [4]:
dataset

,type,posts
0,INFJ,moments sportscenter top ten plays pranks lif...
1,ENTP,finding lack posts alarming sex boring positio...
2,INTP,good one course say know blessing curse absolu...
3,INTJ,dear enjoyed conversation day esoteric gabbin...
4,ENTJ,fired another silly misconception approaching ...
...,...,...
8670,ISFP,always think cats fi doms reason websites beco...
8671,ENFP,thread already exists someplace else heck dele...
8672,INTP,many questions things would take purple pill p...
8673,INFP,conflicted right comes wanting children honest...


In [5]:
dataset_shuffled = dataset.sample(frac=1)

In [6]:
dataset_shuffled

,type,posts
5101,INTJ,thanks reply skeptical makes sense frankly bit...
3216,INTJ,realisation shorts liner difficult put would a...
5390,INFJ,known couple guys seem lack better term easy ...
1308,ENTP,approaching would serial killer perspective gu...
7065,INFJ,hello sadly idea remember caanot even begin co...
...,...,...
5043,ENTP,house goal apparently predisposition us like s...
331,INFP,quantity quality love waterhouse work work pro...
690,ESFP,go home farnsworth fry bender although prog...
267,ISTJ,say lot online tests wonder get confused riso ...


In [7]:
x = dataset['posts']
y = dataset['type']

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, shuffle=True, stratify=y, random_state=12)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, shuffle=True, stratify=y_temp, random_state=12)

In [10]:
print(x_train.shape)
print(y_train.shape)
print()

print(x_val.shape)
print(y_val.shape)
print()

print(x_test.shape)
print(y_test.shape)
print()


(5205,)
(5205,)

(1735,)
(1735,)

(1735,)
(1735,)



In [12]:
def to_int(x):
    types = ['INTJ', 'INTP', 'ENTJ', 'ENTP', 'INFJ', 'INFP', 'ENFJ', 'ENFP', 'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ', 'ISTP', 'ISFP', 'ESTP', 'ESFP']
    integers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    for i in range(len(integers)):
        if x == types[i]:
            result = integers[i] 
    return result

def label_encoding(df_y):
    return df_y.apply(to_int)
    
def one_hot_encode(label, num_classes):
    
    encoded_label = np.zeros(num_classes)
    encoded_label[label] = 1
    return encoded_label

In [17]:
encoded_y.shape

torch.Size([10, 30])

In [26]:
tokens['input_ids'].shape

torch.Size([1, 12])

In [30]:
len(input_text.tolist())

10

In [34]:
input_text = x_train[:10]
label = y_train[:10]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=30)

tokens = tokenizer(input_text.tolist(), add_special_tokens=True, truncation=True, padding=True, return_tensors='pt')

y = label_encoding(label)
encoded_labels = y.apply(lambda y: one_hot_encode(label=y, num_classes=30))
encoded_y = torch.tensor(encoded_labels.tolist(), dtype=torch.float)

outputs = model(**tokens, labels=encoded_y)
loss = outputs.loss
loss.backward()

predictions = torch.softmax(outputs.logits, dim=1)
predicted_labels = torch.argmax(predictions, dim=1)

print("predicted_labels: ", predicted_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

predicted_labels:  tensor([ 5,  5, 22,  5,  5,  5,  5,  5,  5,  5])
